In [7]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, HoverTool, ColorBar
from bokeh.palettes import Viridis256
from bokeh.transform import linear_cmap
import json
import geopandas as gpd

output_file("enhanced_geospatial_data.html")
world = gpd.read_file("ne_110m_admin_0_countries.shp") 
world['dummy_metric'] = world.index % 50  

# Converting GeoPandas DataFrame to GeoJSON format
geojson_data = json.loads(world.to_json())
geojson_source = GeoJSONDataSource(geojson=json.dumps(geojson_data))

# Color mapper
color_mapper = linear_cmap(
    field_name="dummy_metric", 
    palette=Viridis256, 
    low=min(world["dummy_metric"]), 
    high=max(world["dummy_metric"])
)

# Bokeh plot
p = figure(
    title="Enhanced Geospatial Data Visualization",
    width=800,
    height=600,
    x_axis_label="Longitude",
    y_axis_label="Latitude",
    toolbar_location="below"
)

# Along with color mapping ,also adding patches
p.patches(
    "xs",
    "ys",
    source=geojson_source,
    fill_color=color_mapper,
    fill_alpha=0.7,
    line_color="white",
    line_width=0.5
)

# Adding interactivity using HoverTool
hover = HoverTool(
    tooltips=[
        ("Country", "@name"), 
        ("Metric Value", "@dummy_metric")
    ]
)
p.add_tools(hover)

# Adding color bar to the metrics
color_bar = ColorBar(
    color_mapper=color_mapper['transform'], 
    width=8, 
    location=(0, 0),
    title="Metric Value"
)
p.add_layout(color_bar, 'right')

show(p)